In [1]:
import requests
from requests.auth import HTTPBasicAuth
import json
from nltk.text import sent_tokenize
from nltk import word_tokenize
from rank_bm25 import BM25Okapi
from utils import remove_brackets

## Document API

In [ ]:
url = "https://guacamole.univ-avignon.fr/dblp1/_search?q=%22Digital%20assistant%22&size=10"
resp = requests.get(url, auth=HTTPBasicAuth('inex', 'qatc2011'), verify=False)

In [ ]:
contents = json.loads(resp.content)

In [ ]:
contents['hits']['hits'][0]['_source'].keys()

In [ ]:
docs = [hit['_source'] for hit in contents['hits']['hits']]

In [ ]:
docs[0]

In [ ]:
sent_tokenize(docs[0]['abstract'])

## BM25

In [2]:
with open('topics/SP12022topics.json') as f_in:
    topics = json.load(f_in)

In [3]:
topic = topics[0]
topic

{'topic_id': 'G01',
 'topic_text': 'Digital assistants like Siri and Alexa entrench gender biases says UN',
 'topic_url': 'https://www.theguardian.com/technology/2019/may/22/digital-voice-assistants-siri-alexa-gender-biases-unesco-says',
 'query_id': 'G01.1',
 'query_text': 'Digital assistant',
 'abstract_url': 'https://guacamole.univ-avignon.fr/dblp1/_search?q="Digital assistant"'}

In [16]:
topic_content_file = 'topics/topic_related_content/topic' + topic['topic_id'] + '.md'
topic_text = topic['topic_text']
abstract_url = topic['abstract_url']
resp = requests.get(abstract_url, auth=HTTPBasicAuth('inex', 'qatc2011'), verify=False)
contents = json.loads(resp.content)
abstracts = [hit['_source']['abstract'] for hit in contents['hits']['hits']]

C:\Users\kerui\miniconda3\envs\Win10\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'guacamole.univ-avignon.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [17]:
docs = []
for abstract in abstracts:
    docs.extend(sent_tokenize(abstract))
tokenized_corpus = [word_tokenize(doc.lower()) for doc in docs]
bm25 = BM25Okapi(tokenized_corpus)

In [18]:
with open(topic_content_file) as f_in:
    topic_content = []
    for line in f_in:
        line = line.strip().lower()
        if line:
            topic_content.extend(word_tokenize(remove_brackets(line.strip())))

In [19]:
title_as_query_scores = bm25.get_scores(word_tokenize(topic_text.lower()))
title_as_query_scores

array([0.91104245, 0.80261701, 0.20949334, 1.00408621, 1.30490518,
       0.91104245, 0.80261701, 0.20949334, 1.00408621, 1.30490518,
       0.87581333, 0.61866356, 0.        , 0.75849266, 0.86174748,
       0.        , 0.        , 0.71087929, 3.58942399, 2.32317842,
       0.20949334, 0.        , 0.89308061, 0.        , 0.        ,
       0.7353412 , 0.        , 2.584723  , 3.26311333, 0.        ,
       0.71087929, 0.7846698 , 0.75849266, 0.96954138, 0.        ,
       0.26359664, 0.        , 0.        , 0.24464239, 0.51817331,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [20]:
article_as_query_scores = bm25.get_scores(topic_content)
article_as_query_scores

array([ 40.00564763,  51.71556647,  61.73998786,  53.95169306,
        51.7821711 ,  40.00564763,  51.71556647,  61.73998786,
        53.95169306,  51.7821711 ,  38.73201225,  37.33893928,
        13.36977693,  59.58939866,  25.37119845,  28.88610269,
        46.2363724 ,  56.91981031,  63.93168605,  68.19961695,
        43.79798526,  64.74918758,  55.04827488,  64.25824217,
        26.38560643,  77.15492727,  29.32169481, 100.71077927,
        74.70768156,  40.70575396,  46.83192013,  52.23135643,
        87.99227928,  64.87256409,  48.51119183,  36.78318696,
        33.81508135,  85.62525659,  68.84870093,  38.88155814,
        13.78795961,  24.7856463 ,  24.7856463 ,  12.28012259,
        28.4376667 ])

In [23]:
print('============== Query title ==============')
print(topic_text)
print('============== Highest ranked passage by title ==============')
print(docs[title_as_query_scores.argmax()])
print('============== Highest ranked passage by content ==============')
print(docs[article_as_query_scores.argmax()])

============== Query title ==============
Digital assistants like Siri and Alexa entrench gender biases says UN
============== Highest ranked passage by title ==============
Digital assistants are emerging to become more prevalent in our daily lives.
============== Highest ranked passage by content ==============
In particular, the ubiquitous access to remote resources is one of the most interesting characteristics achievable by using mobile devices such as Personal Digital Assistants, cellular phones and tablets.
